In [ ]:
from google.colab import drive
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np


In [8]:
IMAGE_SIZE = (224,224)
batch_size = 64
drive.mount('/content/drive')
dir_path = '/content/drive/MyDrive/Projeto_LCDIA'
print(dir_path)
#dataset preparation
dataset = image_dataset_from_directory(
    dir_path,
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    shuffle=True,
    validation_split=0.2,
    subset='training',
    image_size=IMAGE_SIZE,
    batch_size=batch_size,
    seed=4225
)
#validation dataset
validation = dataset = image_dataset_from_directory(
    dir_path,
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    shuffle=True,
    validation_split=0.2,
    subset='validation',
    image_size=IMAGE_SIZE,
    batch_size=batch_size,
    seed=4225
)


--2024-12-19 22:37:51--  https://code-server.dev/install.sh
Resolving code-server.dev (code-server.dev)... 104.21.16.171, 172.67.214.225, 2606:4700:3031::ac43:d6e1, ...
Connecting to code-server.dev (code-server.dev)|104.21.16.171|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/cdr/code-server/main/install.sh [following]
--2024-12-19 22:37:53--  https://raw.githubusercontent.com/cdr/code-server/main/install.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15872 (16K) [text/plain]
Saving to: ‘install.sh.3’

     0K .......... .....                                      100%  102M=0s

2024-12-19 22:37:53 (102 MB/s) - ‘install.sh.3’ saved [15872/15872]

sudo: a terminal is required to read the pa

FileNotFoundError: [Errno 2] No such file or directory: 'code-server'

In [ ]:
for images, labels in dataset.take(1):
    print("Images shape:", images.shape)
    print("Labels:", labels)
#Improting the model
pretrained_model = ResNet50(
    input_shape=(224,224,1), 
    weights=None, 
    include_top=False,
    pooling='max')
model = Sequential()

In [ ]:
#Decide which layers to freeze
#Let's try to freeze all stage 1 layers
#Let's not change pooling as max pooling will be better to extract shapes
#first 4 layers: convolutional, batch normalization, relu activation, max pooling
# for layer in pretrained_model.layers:
#     print(layer.name)

for layer in pretrained_model.layers[:6]:  # Freeze the first 4 layers
    layer.trainable = False
#add to model
model.add(pretrained_model)


In [ ]:
#Data augmentation with ImageDataGenerator

data_augmentations = Sequential([
    layers.RandomFlip("horizontal"),  # Flip images horizontally
    layers.RandomRotation(0.3),  # Rotate images by up to 20%
    layers.RandomZoom(0.3),  # Zoom images by up to 20%
    layers.RandomContrast(0.3)  # Apply random contrast changes
])
#using automated augmentation
dataset = dataset.map(lambda x,y: (data_augmentations(x),y))
dataset = dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)



In [ ]:
#Adding final layers
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(4, activation='softmax'))

#Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',        # Monitor validation loss
    patience=5,                # Stop after 5 epochs with no improvement
    restore_best_weights=True  # Restore the best model weights
)

#Let's use learning rate 0.002
model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(dataset, validation_data=validation, epochs=40, callbacks=[early_stopping])

In [ ]:
#Evaluation
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['validation_accuracy'])
plt.axis(ymin=0.4, ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()